## Hello : )

### This project is build to discovery some insights of airbnb acomodation in Seattle
 
for now, we want to awnser some questions like:
- how much I need to rent a acomodation in seattle ? 
- what month of year, we have more available acomodation ?
- what's the characterisct that rise the price ?

but first, you need to install some libs
let's go? 

by: nathalia cardoso / july 2022
kaggle
https://www.kaggle.com/code/nathycardoso/seattle-airbnb-open-data

In [52]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

## your dataset it's to big , and I want to see each row
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import matplotlib.pyplot as plt

#https://plotly.com/python/bar-charts/
import plotly.express as px

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder



### Datasets imports and overview

In [23]:
calendar_df = pd.read_csv('calendar.csv')
calendar_df.head()

,listing_id,date,available,price
0,241032,2016-01-04,t,$85.00
1,241032,2016-01-05,t,$85.00
2,241032,2016-01-06,f,NaN
3,241032,2016-01-07,f,NaN
4,241032,2016-01-08,f,NaN


In [24]:
listings_df = pd.read_csv('listings.csv')
listings_df.head(2)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,241032,https://www.airbnb.com/rooms/241032,20160104002432,2016-01-04,Stylish Queen Anne Apartment,NaN,Make your self at home in this charming one-be...,Make your self at home in this charming one-be...,none,NaN,NaN,NaN,NaN,NaN,https://a1.muscache.com/ac/pictures/67560560/c...,NaN,956883,https://www.airbnb.com/users/show/956883,Maija,2011-08-11,"Seattle, Washington, United States","I am an artist, interior designer, and run a s...",within a few hours,96%,100%,f,https://a0.muscache.com/ac/users/956883/profil...,https://a0.muscache.com/ac/users/956883/profil...,Queen Anne,3.0,3.0,"['email', 'phone', 'reviews', 'kba']",t,t,"Gilman Dr W, Seattle, WA 98119, United States",Queen Anne,West Queen Anne,Queen Anne,Seattle,WA,98119,Seattle,"Seattle, WA",US,United States,47.636289,-122.371025,t,Apartment,Entire home/apt,4,1.0,1.0,1.0,Real Bed,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",NaN,$85.00,NaN,NaN,NaN,NaN,2,$5.00,1,365,4 weeks ago,t,14,41,71,346,2016-01-04,207,2011-11-01,2016-01-02,95.0,10.0,10.0,10.0,10.0,9.0,10.0,f,NaN,WASHINGTON,f,moderate,f,f,2,4.07
1,953595,https://www.airbnb.com/rooms/953595,20160104002432,2016-01-04,Bright & Airy Queen Anne Apartment,Chemically sensitive? We've removed the irrita...,"Beautiful, hypoallergenic apartment in an extr...",Chemically sensitive? We've removed the irrita...,none,"Queen Anne is a wonderful, truly functional vi...",What's up with the free pillows? Our home was...,"Convenient bus stops are just down the block, ...",https://a0.muscache.com/ac/pictures/14409893/f...,https://a0.muscache.com/im/pictures/14409893/f...,https://a0.muscache.com/ac/pictures/14409893/f...,https://a0.muscache.com/ac/pictures/14409893/f...,5177328,https://www.airbnb.com/users/show/5177328,Andrea,2013-02-21,"Seattle, Washington, United States",Living east coast/left coast/overseas. Time i...,within an hour,98%,100%,t,https://a0.muscache.com/ac/users/5177328/profi...,https://a0.muscache.com/ac/users/5177328/profi...,Queen Anne,6.0,6.0,"['email', 'phone', 'facebook', 'linkedin', 're...",t,t,"7th Avenue West, Seattle, WA 98119, United States",Queen Anne,West Queen Anne,Queen Anne,Seattle,WA,98119,Seattle,"Seattle, WA",US,United States,47.639123,-122.365666,t,Apartment,Entire home/apt,4,1.0,1.0,1.0,Real Bed,"{TV,Internet,""Wireless Internet"",Kitchen,""Free...",NaN,$150.00,"$1,000.00","$3,000.00",$100.00,$40.00,1,$0.00,2,90,today,t,13,13,16,291,2016-01-04,43,2013-08-19,2015-12-29,96.0,10.0,10.0,10.0,10.0,10.0,10.0,f,NaN,WASHINGTON,f,strict,t,t,6,1.48


### First Steps

#### Discovery dataset - EDA

In [25]:
#how many listings we have in this dataset ? 
len(listings_df.id.unique())

3818

In [26]:
listings_df.shape
# there's a lot of columns ( text and numeric )

(3818, 92)

In [27]:
listings_df.describe()

#some columns have null values : example square_feet and license


,id,scrape_id,host_id,host_listings_count,host_total_listings_count,latitude,longitude,accommodates,bathrooms,bedrooms,beds,square_feet,guests_included,minimum_nights,maximum_nights,availability_30,availability_60,availability_90,availability_365,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,calculated_host_listings_count,reviews_per_month
count,3.818000e+03,3.818000e+03,3.818000e+03,3816.000000,3816.000000,3818.000000,3818.000000,3818.000000,3802.000000,3812.000000,3817.000000,97.000000,3818.000000,3818.000000,3818.000000,3818.000000,3818.000000,3818.000000,3818.000000,3818.000000,3171.000000,3160.000000,3165.000000,3160.000000,3167.000000,3163.000000,3162.000000,0.0,3818.000000,3191.000000
mean,5.550111e+06,2.016010e+13,1.578556e+07,7.157757,7.157757,47.628961,-122.333103,3.349398,1.259469,1.307712,1.735394,854.618557,1.672603,2.369303,780.447617,16.786276,36.814825,58.082504,244.772656,22.223415,94.539262,9.636392,9.556398,9.786709,9.809599,9.608916,9.452245,NaN,2.946307,2.078919
std,2.962660e+06,0.000000e+00,1.458382e+07,28.628149,28.628149,0.043052,0.031745,1.977599,0.590369,0.883395,1.139480,671.404893,1.311040,16.305902,1683.589007,12.173637,23.337541,34.063845,126.772526,37.730892,6.606083,0.698031,0.797274,0.595499,0.568211,0.629053,0.750259,NaN,5.893029,1.822348
min,3.335000e+03,2.016010e+13,4.193000e+03,1.000000,1.000000,47.505088,-122.417219,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,20.000000,2.000000,3.000000,2.000000,2.000000,4.000000,2.000000,NaN,1.000000,0.020000
25%,3.258256e+06,2.016010e+13,3.275204e+06,1.000000,1.000000,47.609418,-122.354320,2.000000,1.000000,1.000000,1.000000,420.000000,1.000000,1.000000,60.000000,2.000000,13.000000,28.000000,124.000000,2.000000,93.000000,9.000000,9.000000,10.000000,10.000000,9.000000,9.000000,NaN,1.000000,0.695000
50%,6.118244e+06,2.016010e+13,1.055814e+07,1.000000,1.000000,47.623601,-122.328874,3.000000,1.000000,1.000000,1.000000,750.000000,1.000000,2.000000,1125.000000,20.000000,46.000000,73.000000,308.000000,9.000000,96.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,NaN,1.000000,1.540000
75%,8.035127e+06,2.016010e+13,2.590309e+07,3.000000,3.000000,47.662694,-122.310800,4.000000,1.000000,2.000000,2.000000,1200.000000,2.000000,2.000000,1125.000000,30.000000,59.000000,89.000000,360.000000,26.000000,99.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,NaN,2.000000,3.000000
max,1.034016e+07,2.016010e+13,5.320861e+07,502.000000,502.000000,47.733358,-122.240607,16.000000,8.000000,7.000000,15.000000,3000.000000,15.000000,1000.000000,100000.000000,30.000000,60.000000,90.000000,365.000000,474.000000,100.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,NaN,37.000000,12.150000


# 1) how much I need to rent a acomodation in seattle ? 


In [28]:
#format the column price and transform in a float type
listings_df['price'] = listings_df['price'].str.replace('$','').str.replace(',','').astype(float)

fig = px.histogram(listings_df, x="price")
fig.show()

### 50% of listings cost 100$/ day 


In [29]:
listings_df['price'].describe()

count    3818.000000
mean      127.976166
std        90.250022
min        20.000000
25%        75.000000
50%       100.000000
75%       150.000000
max      1000.000000
Name: price, dtype: float64

### Some neighbourhood like industrial district has a higher price. 


- here, we have some outiler's listings - this can be clean after

In [30]:
fig = px.box(listings_df, x="neighbourhood", y="price")
fig.show()


In [114]:
neighbourhood = listings_df.groupby('neighbourhood')['price'].median() 
neighbourhood = neighbourhood.to_frame().reset_index()
top_neighbourhoods ='|'.join(list(neighbourhood[neighbourhood.price > 100]['neighbourhood']))

# 2) what month of year, we have more available acomodation ?

In [31]:
calendar_df.head()


,listing_id,date,available,price
0,241032,2016-01-04,t,$85.00
1,241032,2016-01-05,t,$85.00
2,241032,2016-01-06,f,NaN
3,241032,2016-01-07,f,NaN
4,241032,2016-01-08,f,NaN


In [32]:
#the number of total houses change along the year ? NO - so we have a fixed number of houses in this dataset
total = calendar_df.groupby("date").listing_id.nunique()
total.unique()

array([3818], dtype=int64)

In [33]:
available_houses = round((calendar_df[calendar_df['available'] == 't'].groupby("date").listing_id.nunique() / total) * 100,0)

#transform series to dataframe
available_houses = available_houses.to_frame().reset_index()

#plot result in a bar chart using ploty
fig = px.bar(available_houses, x='date', y='listing_id')
fig.show()

### look by day it's hard to see , so let's take a look in availability by month

## The median and average avalabity by month is close

In [34]:
#month mean availability

#first, transform date column 
available_houses['date'] = pd.to_datetime(available_houses['date'], errors='coerce')

#format date to year-month format
available_houses['month'] = available_houses['date'].dt.strftime('%m')

#describe for each month values
available_houses.groupby('month').describe()
#mean and median are close so we can use median for easy undestanding 

listing_id                                                    
           count       mean       std   min    25%   50%   75%   max
month                                                               
01          30.0  56.733333  7.104798  45.0  52.25  58.0  60.0  77.0
02          29.0  66.206897  2.304953  62.0  64.00  66.0  68.0  70.0
03          31.0  70.838710  0.734701  70.0  70.00  71.0  71.0  72.0
04          30.0  66.333333  1.582955  64.0  66.00  66.0  67.0  73.0
05          31.0  67.548387  0.505879  67.0  67.00  68.0  68.0  68.0
06          30.0  67.333333  0.711159  66.0  67.00  67.0  68.0  69.0
07          31.0  62.870968  0.340777  62.0  63.00  63.0  63.0  63.0
08          31.0  64.548387  0.994609  63.0  64.00  65.0  65.0  66.0
09          30.0  67.366667  0.764890  66.0  67.00  67.0  68.0  69.0
10          31.0  69.580645  0.501610  69.0  69.00  70.0  70.0  70.0
11          30.0  71.366667  0.614948  70.0  71.00  71.0  72.0  72.0
12          31.0  73.548387  0.623897  72.0  73.00  74.0  74.0  75.0

## In the end of year, the availabity grow.

In [35]:
#get the mean of each month 
month_available_houses = (round(available_houses.groupby('month').listing_id.mean(),0)).to_frame().reset_index()

#plot value by month

fig = px.bar(month_available_houses, x='month', y='listing_id')
fig.show()

In [36]:
#the weather can be a factor how impacts in houses availabity ?
#you can found more in this website https://weatherspark.com/y/913/Average-Weather-in-Seattle-Washington-United-States-Year-Round

weather = {'month': ['01','02','03','04','05','06','07','08','09','10','11','12'],
           'High': [48,51,55,60,66,72,77,77,71,61,52,47],
           'Temp': [42,44,47,51,57,62,66,67,62,53,46,41],
           'Low': [38,39,42,45,50,55,58,58,54,48,42,38]
          }

weather_df = pd.DataFrame(data=weather)
weather_df.head()

,month,High,Temp,Low
0,01,48,42,38
1,02,51,44,39
2,03,55,47,42
3,04,60,51,45
4,05,66,57,50


## the weather can be a factor how impacts in houses availabity ?

- its possible, as you can see in the graph below on october and foward the temperature fall and the count os listing availiabilty grow. 
- to make sure this is a strong relation, mora data of weather along de time needs to be analyse. ( in this project we will stop here )

In [37]:
month_available_houses['Temp'] = weather['Temp']
month_available_houses.head()

fig = px.scatter_matrix(month_available_houses)
fig.show()


In [38]:
month_available_houses.corr()
#existing a negative correlation but is not strong

,listing_id,Temp
listing_id,1.000000,-0.199819
Temp,-0.199819,1.000000


# 3) what's the characteristic that rise the price ?

In [115]:
#change some datatypes
df_dummies = listings_df.copy()

df_dummies['host_is_superhost'] = df_dummies['host_is_superhost'].replace('t',1).replace('f',0)
df_dummies['host_identity_verified'] = df_dummies['host_identity_verified'].replace('t',1).replace('f',0)
df_dummies['host_response_rate'] = df_dummies['host_response_rate'].str.replace('%','')
df_dummies['host_acceptance_rate'] = df_dummies['host_acceptance_rate'].str.replace('%','')
df_dummies = df_dummies.join(pd.get_dummies(df_dummies['property_type'],prefix='property_type'))
df_dummies = df_dummies.join(pd.get_dummies(df_dummies['cancellation_policy'],prefix='cancellation_policy'))
df_dummies = df_dummies.join(pd.get_dummies(df_dummies['room_type'],prefix='room_type'))
#df_dummies = df_dummies.join(pd.get_dummies(df_dummies['neighbourhood'],prefix='neighbourhood'))
df_dummies['top_neighbourhoods'] = (df_dummies['neighbourhood'].str.contains(top_neighbourhoods)*1).astype(np.float32)

df_dummies.head(2)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,property_type_Apartment,property_type_Bed & Breakfast,property_type_Boat,property_type_Bungalow,property_type_Cabin,property_type_Camper/RV,property_type_Chalet,property_type_Condominium,property_type_Dorm,property_type_House,property_type_Loft,property_type_Other,property_type_Tent,property_type_Townhouse,property_type_Treehouse,property_type_Yurt,cancellation_policy_flexible,cancellation_policy_moderate,cancellation_policy_strict,room_type_Entire home/apt,room_type_Private room,room_type_Shared room,top_neighbourhoods
0,241032,https://www.airbnb.com/rooms/241032,20160104002432,2016-01-04,Stylish Queen Anne Apartment,NaN,Make your self at home in this charming one-be...,Make your self at home in this charming one-be...,none,NaN,NaN,NaN,NaN,NaN,https://a1.muscache.com/ac/pictures/67560560/c...,NaN,956883,https://www.airbnb.com/users/show/956883,Maija,2011-08-11,"Seattle, Washington, United States","I am an artist, interior designer, and run a s...",within a few hours,96,100,0.0,https://a0.muscache.com/ac/users/956883/profil...,https://a0.muscache.com/ac/users/956883/profil...,Queen Anne,3.0,3.0,"['email', 'phone', 'reviews', 'kba']",t,1.0,"Gilman Dr W, Seattle, WA 98119, United States",Queen Anne,West Queen Anne,Queen Anne,Seattle,WA,98119,Seattle,"Seattle, WA",US,United States,47.636289,-122.371025,t,Apartment,Entire home/apt,4,1.0,1.0,1.0,Real Bed,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",NaN,85.0,NaN,NaN,NaN,NaN,2,$5.00,1,365,4 weeks ago,t,14,41,71,346,2016-01-04,207,2011-11-01,2016-01-02,95.0,10.0,10.0,10.0,10.0,9.0,10.0,f,NaN,WASHINGTON,f,moderate,f,f,2,4.07,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1.0
1,953595,https://www.airbnb.com/rooms/953595,20160104002432,2016-01-04,Bright & Airy Queen Anne Apartment,Chemically sensitive? We've removed the irrita...,"Beautiful, hypoallergenic apartment in an extr...",Chemically sensitive? We've removed the irrita...,none,"Queen Anne is a wonderful, truly functional vi...",What's up with the free pillows? Our home was...,"Convenient bus stops are just down the block, ...",https://a0.muscache.com/ac/pictures/14409893/f...,https://a0.muscache.com/im/pictures/14409893/f...,https://a0.muscache.com/ac/pictures/14409893/f...,https://a0.muscache.com/ac/pictures/14409893/f...,5177328,https://www.airbnb.com/users/show/5177328,Andrea,2013-02-21,"Seattle, Washington, United States",Living east coast/left coast/overseas. Time i...,within an hour,98,100,1.0,https://a0.muscache.com/ac/users/5177328/profi...,https://a0.muscache.com/ac/users/5177328/profi...,Queen Anne,6.0,6.0,"['email', 'phone', 'facebook', 'linkedin', 

In [129]:
fig = px.scatter_matrix(df_dummies[['price','accommodates','bathrooms','bedrooms','beds','guests_included','host_response_rate']])
fig.show()

In [116]:
## select only numeric columns 
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64','uint8']
df_numeric_values = df_dummies.select_dtypes(include=numerics)

df_numeric_values

,id,scrape_id,host_id,host_is_superhost,host_listings_count,host_total_listings_count,host_identity_verified,latitude,longitude,accommodates,bathrooms,bedrooms,beds,square_feet,price,guests_included,minimum_nights,maximum_nights,availability_30,availability_60,availability_90,availability_365,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,calculated_host_listings_count,reviews_per_month,property_type_Apartment,property_type_Bed & Breakfast,property_type_Boat,property_type_Bungalow,property_type_Cabin,property_type_Camper/RV,property_type_Chalet,property_type_Condominium,property_type_Dorm,property_type_House,property_type_Loft,property_type_Other,property_type_Tent,property_type_Townhouse,property_type_Treehouse,property_type_Yurt,cancellation_policy_flexible,cancellation_policy_moderate,cancellation_policy_strict,room_type_Entire home/apt,room_type_Private room,room_type_Shared room,top_neighbourhoods
0,241032,20160104002432,956883,0.0,3.0,3.0,1.0,47.636289,-122.371025,4,1.0,1.0,1.0,NaN,85.0,2,1,365,14,41,71,346,207,95.0,10.0,10.0,10.0,10.0,9.0,10.0,NaN,2,4.07,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1.0
1,953595,20160104002432,5177328,1.0,6.0,6.0,1.0,47.639123,-122.365666,4,1.0,1.0,1.0,NaN,150.0,1,2,90,13,13,16,291,43,96.0,10.0,10.0,10.0,10.0,10.0,10.0,NaN,6,1.48,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1.0
2,3308979,20160104002432,16708587,0.0,2.0,2.0,1.0,47.629724,-122.369483,11,4.5,5.0,7.0,NaN,975.0,10,4,30,1,6,17,220,20,97.0,10.0,10.0,10.0,10.0,10.0,10.0,NaN,2,1.15,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,1.0
3,7421966,20160104002432,9851441,0.0,1.0,1.0,1.0,47.638473,-122.369279,3,1.0,0.0,2.0,NaN,100.0,1,1,1125,0,0,0,143,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1.0
4,278830,20160104002432,1452570,0.0,2.0,2.0,1.0,47.632918,-122.372471,6,2.0,3.0,3.0,NaN,450.0,6,1,1125,30,60,90,365,38,92.0,9.0,9.0,10.0,10.0,9.0,9.0,NaN,1,0.89,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,1.0
5,5956968,20160104002432,326758,0.0,1.0,1.0,1.0,47.630525,-122.366174,2,1.0,1.0,1.0,NaN,120.0,1,1,6,0,0,27,302,17,95.0,10.0,10.0,10.0,10.0,10.0,10.0,NaN,1,2.45,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,1.0
6,1909058,20160104002432,2497928,1.0,1.0,1.0,1.0,47.636605,-122.368519,2,1.0,1.0,1.0,NaN,80.0,1,3,14,20,49,71,346,58,99.0,10.0,10.0,10.0,10.0,10.0,10.0,NaN,1,2.46,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1.0
7,856550,20160104002432,4016632,1.0,5.0,5.0,1.0,47.640161,-122.375856,2,1.0,1.0,1.0,NaN,60.0,1,2,7,22,42,42,100,173,97.0,10.0,10.0,10.0,10.0,9.0,10.0,NaN,5,4.73,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1.0
8,4948745,20160104002432,2166277,0.0,1.0,1.0,1.0,47.632410,-122.357216,2,1.0,1.0,1.0,NaN,90.0,1,3,1125,6,36,66,341,8,97.0,10.0,9.0,10.0,9.0,10.0,10.0,NaN,1,1.22,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1.0
9,2493658,20160104002432,5177328,1.0,6.0,6.0,1.0,47.637492,-122.366889,4,1.0,1.0,1.0,NaN,150.0,1,2,365,6,19,49,324,32,97.0,10.0,10.0,10.0,10.0,10.0,9.0,NaN,6,1.55,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1.0


In [118]:
#check null values for each column 
#license and square_feet is all null - so we can remove this columns 
#review we will drop for now ... new houses dosen`t have review until is rent. 

((df_numeric_values.isna().sum() / len(df_numeric_values))*100).sort_values(ascending=False).head(11)


license                        100.000000
square_feet                     97.459403
review_scores_accuracy          17.234154
review_scores_checkin           17.234154
review_scores_value             17.181771
review_scores_location          17.155579
review_scores_cleanliness       17.103195
review_scores_communication     17.050812
review_scores_rating            16.946045
reviews_per_month               16.422211
top_neighbourhoods              10.895757
dtype: float64

In [119]:
#drop some collumns 
df_cluster = df_numeric_values.drop(['number_of_reviews','maximum_nights','minimum_nights','id','host_id','license','square_feet','host_total_listings_count','availability_365','availability_90','host_listings_count','calculated_host_listings_count','availability_60','availability_30','review_scores_checkin','review_scores_accuracy','review_scores_value','review_scores_location','review_scores_cleanliness','review_scores_communication','review_scores_rating','reviews_per_month','latitude','longitude'],axis=1)

df_cluster.head()

,scrape_id,host_is_superhost,host_identity_verified,accommodates,bathrooms,bedrooms,beds,price,guests_included,property_type_Apartment,property_type_Bed & Breakfast,property_type_Boat,property_type_Bungalow,property_type_Cabin,property_type_Camper/RV,property_type_Chalet,property_type_Condominium,property_type_Dorm,property_type_House,property_type_Loft,property_type_Other,property_type_Tent,property_type_Townhouse,property_type_Treehouse,property_type_Yurt,cancellation_policy_flexible,cancellation_policy_moderate,cancellation_policy_strict,room_type_Entire home/apt,room_type_Private room,room_type_Shared room,top_neighbourhoods
0,20160104002432,0.0,1.0,4,1.0,1.0,1.0,85.0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1.0
1,20160104002432,1.0,1.0,4,1.0,1.0,1.0,150.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1.0
2,20160104002432,0.0,1.0,11,4.5,5.0,7.0,975.0,10,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,1.0
3,20160104002432,0.0,1.0,3,1.0,0.0,2.0,100.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1.0
4,20160104002432,0.0,1.0,6,2.0,3.0,3.0,450.0,6,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,1.0


In [120]:
## all the rest off null values I will replace if mode of each column 
df_cluster_no_na_values = df_cluster.fillna(df_cluster.mode())
df_cluster_no_na_values['top_neighbourhoods'].fillna(0,inplace=True)

df_cluster_no_na_values[df_cluster_no_na_values.select_dtypes(np.float64).columns] = df_cluster_no_na_values.select_dtypes(np.float64).astype(np.float32)
df_cluster_no_na_values[df_cluster_no_na_values.select_dtypes(np.int64).columns] = df_cluster_no_na_values.select_dtypes(np.int64).astype(np.float32)
df_cluster_no_na_values = df_cluster_no_na_values.dropna()
df_cluster_no_na_values.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3794 entries, 0 to 3817
Data columns (total 32 columns):
scrape_id                        3794 non-null float32
host_is_superhost                3794 non-null float32
host_identity_verified           3794 non-null float32
accommodates                     3794 non-null float32
bathrooms                        3794 non-null float32
bedrooms                         3794 non-null float32
beds                             3794 non-null float32
price                            3794 non-null float32
guests_included                  3794 non-null float32
property_type_Apartment          3794 non-null uint8
property_type_Bed & Breakfast    3794 non-null uint8
property_type_Boat               3794 non-null uint8
property_type_Bungalow           3794 non-null uint8
property_type_Cabin              3794 non-null uint8
property_type_Camper/RV          3794 non-null uint8
property_type_Chalet             3794 non-null uint8
property_type_Condominium  

### Feature importance to choice how feature we will use in linear regression

the feature that's means explain de price is 
- accommodates
- guests_included
- bathrooms
- beds
- bedrooms
- host_identity_verified
- host_is_superhost
- property_type_House


In [125]:
#feature importance - quais colunas são mais importantes para determinar o preço 
#não precisa normalizar para colocar na arvore 

X = df_cluster_no_na_values.drop('price',axis=1)
y = df_cluster_no_na_values['price']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=42)

# feature_names = [f"feature {i}" for i in range(X.shape[1])]
feature_names = [f"{i}" for i in X.columns]
forest = RandomForestClassifier(random_state=0)
forest.fit(X_train, y_train)


importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)

forest_importances = pd.Series(importances, index=feature_names)

forest_importances_df = forest_importances.to_frame().reset_index()
forest_importances_df.columns = ['name','value']
forest_importances_df.sort_values(by='value',ascending=False).head(20).reset_index()

C:\Users\nathy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.



,index,name,value
0,3,accommodates,0.160738
1,7,guests_included,0.151783
2,4,bathrooms,0.107693
3,6,beds,0.100217
4,30,top_neighbourhoods,0.081884
5,5,bedrooms,0.074554
6,2,host_identity_verified,0.071595
7,1,host_is_superhost,0.062806
8,17,property_type_House,0.025762
9,8,property_type_Apartment,0.024370


In [122]:
def linearregression(i,df_cluster_no_na_values,forest_importances_df):
    columns = list(forest_importances_df.sort_values(by='value',ascending=False).reset_index().head(i+1)['name'])

    X = df_cluster_no_na_values[columns]
    X = X.dropna()
    y = df_cluster_no_na_values['price']

    X_train_lr, X_test_lr, y_train_lr, y_test_lr = train_test_split(X, y, test_size=.3, random_state=42)

    # #linear regression 
    model = make_pipeline(StandardScaler(), LinearRegression())
    model.fit(X_train_lr, y_train_lr)
    train = model.score(X_train_lr, y_train_lr)
    test = model.score(X_test_lr,y_test_lr)
    return [train,test]

In [124]:
loop_result = {'i': [],
           'train': [],
           'test': []
          }

loop_result = pd.DataFrame(data=loop_result)



for i in range(20):
    result =linearregression(i,df_cluster_no_na_values,forest_importances_df)
    loop_result = loop_result.append({'i': i,'train':result[0],'test':result[1]}, ignore_index=True)

loop_result

C:\Users\nathy\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning:

Data with input dtype float32 were all converted to float64 by StandardScaler.

C:\Users\nathy\Anaconda3\lib\site-packages\sklearn\base.py:465: DataConversionWarning:

Data with input dtype float32 were all converted to float64 by StandardScaler.

C:\Users\nathy\Anaconda3\lib\site-packages\sklearn\pipeline.py:511: DataConversionWarning:

Data with input dtype float32 were all converted to float64 by StandardScaler.

C:\Users\nathy\Anaconda3\lib\site-packages\sklearn\pipeline.py:511: DataConversionWarning:

Data with input dtype float32 were all converted to float64 by StandardScaler.

C:\Users\nathy\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning:

Data with input dtype float32 were all converted to float64 by StandardScaler.

C:\Users\nathy\Anaconda3\lib\site-packages\sklearn\base.py:465: DataConversionWarning:

Data with input dtype float32 were a

C:\Users\nathy\Anaconda3\lib\site-packages\sklearn\base.py:465: DataConversionWarning:

Data with input dtype uint8, float32 were all converted to float64 by StandardScaler.

C:\Users\nathy\Anaconda3\lib\site-packages\sklearn\pipeline.py:511: DataConversionWarning:

Data with input dtype uint8, float32 were all converted to float64 by StandardScaler.

C:\Users\nathy\Anaconda3\lib\site-packages\sklearn\pipeline.py:511: DataConversionWarning:

Data with input dtype uint8, float32 were all converted to float64 by StandardScaler.

C:\Users\nathy\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning:

Data with input dtype uint8, float32 were all converted to float64 by StandardScaler.

C:\Users\nathy\Anaconda3\lib\site-packages\sklearn\base.py:465: DataConversionWarning:

Data with input dtype uint8, float32 were all converted to float64 by StandardScaler.

C:\Users\nathy\Anaconda3\lib\site-packages\sklearn\pipeline.py:511: DataConversionWarning:

Data with i

,i,train,test
0,0.0,0.420202,0.436805
1,1.0,0.425255,0.433332
2,2.0,0.466242,0.462877
3,3.0,0.467850,0.459716
4,4.0,0.483127,0.476810
5,5.0,0.496778,0.509115
6,6.0,0.496786,0.508809
7,7.0,0.497168,0.508702
8,8.0,0.508262,0.516974
9,9.0,0.508274,0.516696


### result of linear regression

- the first 5 features show the best value of trains and teste
- the value of test is close to train dataset ( no overfitting )
- but the value of model in general is not so good ( ~50 % of precision in predict price )
- we can improve de model in many ways, like found another features, clustering ou implement PCA e etc